In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

json = {
    "date": "日期",
    "snow_depth_water_equivalent_mean": "平均雪深水当量",
    "surface_net_solar_radiation_mean": "平均地表净太阳辐射", 
    "surface_net_thermal_radiation_mean": "平均地表净热辐射", 
    "surface_pressure_mean": "平均地表气压", 
    "temperature_2m_mean": "平均2米高度温度", 
    "dewpoint_temperature_2m_mean": "平均2米高度露点温度", 
    "u_component_of_wind_10m_mean": "平均10米高度风速 u 分量", 
    "v_component_of_wind_10m_mean": "平均10米高度风速 v 分量", 
    "volumetric_soil_water_layer_1_mean": "体积土壤水层1平均含水量", 
    "volumetric_soil_water_layer_2_mean": "体积土壤水层2平均含水量", 
    "volumetric_soil_water_layer_3_mean": "体积土壤水层3平均含水量", 
    "volumetric_soil_water_layer_4_mean": "体积土壤水层4平均含水量", 
    "snow_depth_water_equivalent_min": "最小雪深水当量", 
    "surface_net_solar_radiation_min": "最小地表净太阳辐射", 
    "surface_net_thermal_radiation_min": "最小地表净热辐射", 
    "surface_pressure_min": "最小地表气压", 
    "temperature_2m_min": "最小2米高度温度", 
    "dewpoint_temperature_2m_min": "最小2米高度露点温度", 
    "u_component_of_wind_10m_min": "最小10米高度风速 u 分量", 
    "v_component_of_wind_10m_min": "最小10米高度风速 v 分量", 
    "volumetric_soil_water_layer_1_min": "体积土壤水层1最小含水量", 
    "volumetric_soil_water_layer_2_min": "体积土壤水层2最小含水量", 
    "volumetric_soil_water_layer_3_min": "体积土壤水层3最小含水量", 
    "volumetric_soil_water_layer_4_min": "体积土壤水层4最小含水量", 
    "snow_depth_water_equivalent_max": "最大雪深水当量", 
    "surface_net_solar_radiation_max": "最大地表净太阳辐射", 
    "surface_net_thermal_radiation_max": "最大地表净热辐射", 
    "surface_pressure_max": "最大地表气压", 
    "temperature_2m_max": "最大2米高度温度", 
    "dewpoint_temperature_2m_max": "最大2米高度露点温度", 
    "u_component_of_wind_10m_max": "最大10米高度风速 u 分量", 
    "v_component_of_wind_10m_max": "最大10米高度风速 v 分量", 
    "volumetric_soil_water_layer_1_max": "体积土壤水层1最大含水量", 
    "volumetric_soil_water_layer_2_max": "体积土壤水层2最大含水量", 
    "volumetric_soil_water_layer_3_max": "体积土壤水层3最大含水量", 
    "volumetric_soil_water_layer_4_max": "体积土壤水层4最大含水量", 
    "total_precipitation_sum": "总降水量", 
    "potential_evaporation_sum": "潜在蒸发量", 
    "streamflow": "流量"
}

# ========== 1. 文档加载 ==========
with open("data/series_test.txt", "r", encoding="utf-8") as f:
    docs = [line.strip() for line in f if line.strip()]

# ========== 2. 向量索引 ==========
print("正在加载 embedding 模型...")
embed_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embed_model.encode(docs, normalize_embeddings=True)

dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)
print(f"已向量化 {len(docs)} 条文档")

# ========== 3. 加载本地 Qwen 模型 ==========
model_path = "./models/Qwen2-1.5B"  

print("正在加载 Qwen 模型...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()
print("模型加载完成")

# ========== 4. 定义检索函数 ==========
def retrieve_context(query, k=3):
    q_vec = embed_model.encode([query], normalize_embeddings=True)
    D, I = index.search(np.array(q_vec), k)
    context = "\n".join([f"[{i+1}] {docs[j]}" for i, j in enumerate(I[0])])
    return context

# ========== 5. 定义 RAG 推理 ==========
def rag_ask(query, k=3, max_new_tokens=2048):
    context = retrieve_context(query, k)
    prompt = f"""你是一个洪水预测助手，你需要根据站点采集的历史数据{context}来进行流量预测，需要注意的是历史数据包含多种属性{json}，你需要发掘不同属性对流量的贡献规律，历史数据包含在资料中。
        几个简单的案例：
            历史数据1：1982-04-01,62.1,145.3,-55.13,98.73,5.99,0.92,3.45,2.11,0.35,0.35,0.32,0.28,56.49,0.0,-93.89,98.54,0.75,-3.96,0.66,0.04,0.32,0.34,0.3,0.28,72.31,438.99,4.1,99.33,9.13,6.54,5.51,5.68,0.38,0.37,0.33,0.29,6.17,21.67,14.57
                    1982-04-02,55.96,170.22,-87.62,99.85,-0.13,-7.95,3.15,-2.32,0.3,0.32,0.33,0.29,55.34,0.0,-109.5,99.04,-2.83,-11.23,1.88,-4.16,0.29,0.3,0.33,0.29,56.46,531.36,-55.78,100.7,2.24,-3.34,4.16,-0.25,0.31,0.33,0.33,0.29,0.01,24.59,15.82
                    1982-04-03,55.04,166.99,-63.11,100.25,-0.58,-8.43,-0.92,0.93,0.28,0.29,0.32,0.29,54.44,0.0,-108.17,98.98,-5.38,-14.68,-4.39,-2.04,0.28,0.28,0.32,0.29,56.08,587.37,-7.47,100.79,3.72,-1.25,1.5,3.51,0.29,0.3,0.33,0.3,1.85,18.63,12.91
                    1982-04-04,53.26,62.59,-15.16,97.53,3.32,1.47,0.55,1.55,0.33,0.32,0.32,0.3,50.11,0.0,-38.36,96.88,-0.91,-4.58,-5.55,-1.9,0.28,0.28,0.32,0.3,57.8,239.62,4.11,98.69,6.61,6.58,4.51,4.75,0.36,0.33,0.32,0.3,17.64,6.84,14.57
                    1982-04-05,50.92,146.62,-64.3,98.8,-2.93,-10.15,3.87,-1.81,0.3,0.32,0.32,0.3,50.54,0.0,-103.44,97.93,-6.5,-15.13,3.11,-2.99,0.29,0.3,0.32,0.3,51.26,564.72,-25.14,99.57,-0.04,-5.37,4.6,0.21,0.31,0.33,0.32,0.3,1.08,21.24,15.4
                    1982-04-06,52.52,142.81,-56.89,98.85,-5.07,-12.63,-0.57,-2.27,0.28,0.29,0.32,0.3,50.2,0.0,-102.87,97.18,-10.04,-16.35,-3.33,-6.48,0.28,0.29,0.32,0.3,66.16,603.78,-10.95,99.66,-0.57,-7.0,3.25,0.56,0.29,0.3,0.32,0.3,16.23,15.52,9.99
                    1982-04-07,75.4,56.9,-16.77,97.08,-5.77,-9.56,2.57,-5.8,0.27,0.28,0.31,0.3,68.19,0.0,-21.51,96.75,-7.0,-11.3,-2.38,-6.98,0.27,0.27,0.31,0.3,79.18,210.65,-12.36,97.56,-4.61,-7.35,4.75,-3.95,0.27,0.29,0.32,0.3,13.6,13.61,7.08
                    1982-04-08,79.52,158.12,-51.49,98.05,-5.56,-11.56,4.32,-2.66,0.26,0.27,0.31,0.3,79.34,0.0,-76.91,97.56,-8.32,-13.12,3.7,-4.26,0.26,0.26,0.31,0.3,79.7,553.48,-22.64,98.6,-1.96,-10.59,4.96,-1.28,0.27,0.27,0.31,0.3,0.96,18.54,5.62
                    1982-04-09,78.96,182.23,-81.62,98.69,-2.64,-9.49,3.28,-0.42,0.26,0.26,0.3,0.3,78.52,0.0,-97.0,98.5,-6.98,-12.3,0.77,-1.96,0.25,0.26,0.3,0.3,79.33,567.29,-62.74,98.85,3.06,-4.97,4.7,0.53,0.26,0.26,0.31,0.3,0.05,20.05,5.0
                    1982-04-10,78.22,135.11,-69.01,99.1,-1.99,-7.48,3.38,-0.88,0.25,0.26,0.3,0.3,77.82,0.0,-90.4,98.64,-7.36,-9.68,2.25,-1.95,0.25,0.25,0.3,0.3,78.5,385.08,-31.27,99.82,3.03,-4.31,5.07,0.23,0.25,0.26,0.3,0.3,0.07,17.18,4.58
            下一天即1982-04-11的流量：4.79
            历史数据2：1982-04-12,72.28,144.85,-60.69,99.67,1.66,-3.14,0.7,-1.5,0.28,0.26,0.29,0.3,69.98,0.0,-90.64,99.29,-2.83,-5.15,-1.53,-2.59,0.26,0.26,0.29,0.3,74.17,442.56,-21.65,100.3,7.08,-1.11,2.33,0.01,0.3,0.27,0.29,0.3,0.34,14.91,5.0
                    1982-04-13,66.93,122.66,-51.06,99.93,2.95,-1.54,0.34,2.85,0.3,0.27,0.29,0.3,60.66,0.0,-103.5,98.5,-2.34,-6.51,-1.62,-0.82,0.27,0.27,0.29,0.3,69.94,475.23,3.63,100.51,6.85,5.67,2.36,5.76,0.34,0.3,0.29,0.3,3.52,14.06,5.2
                    1982-04-14,55.12,165.64,-72.06,99.22,4.11,-3.0,4.59,-0.77,0.32,0.31,0.29,0.3,52.78,0.0,-98.3,98.36,0.52,-6.51,2.15,-3.3,0.3,0.3,0.29,0.3,59.72,521.56,2.18,100.41,6.64,5.23,7.26,4.17,0.34,0.32,0.29,0.3,0.59,33.58,5.41
                    1982-04-15,51.31,213.57,-101.32,101.17,2.59,-4.94,1.52,-0.04,0.3,0.3,0.29,0.3,49.6,0.0,-116.84,100.49,-2.41,-7.26,1.19,-2.16,0.29,0.3,0.29,0.3,52.74,634.66,-86.0,101.48,8.01,-1.8,2.04,2.18,0.31,0.31,0.29,0.3,0.0,21.44,5.83
                    1982-04-16,44.36,178.06,-73.58,101.22,6.14,1.21,1.65,2.75,0.31,0.31,0.29,0.3,37.03,0.0,-87.0,100.9,0.65,-1.72,1.15,1.85,0.29,0.3,0.29,0.3,49.59,530.18,-52.12,101.46,11.56,3.55,2.31,3.88,0.34,0.32,0.29,0.3,0.03,18.14,6.24
                    1982-04-17,26.86,159.52,-47.37,100.25,10.28,7.59,2.2,3.43,0.34,0.33,0.3,0.3,14.79,0.0,-72.76,99.57,5.54,3.62,1.69,2.02,0.32,0.32,0.29,0.3,36.6,523.13,-5.73,100.85,14.53,10.05,2.65,4.42,0.35,0.34,0.31,0.3,1.32,11.55,6.66
                    1982-04-18,7.53,116.9,-38.49,99.27,9.37,4.4,3.3,2.12,0.32,0.33,0.32,0.31,4.73,0.0,-99.45,98.99,2.65,-5.96,1.67,-1.22,0.3,0.32,0.31,0.3,13.93,604.44,3.94,99.95,13.15,10.4,4.77,4.98,0.34,0.34,0.32,0.31,2.01,19.56,7.91
                    1982-04-19,4.18,253.4,-97.66,100.45,5.85,-3.78,3.28,0.98,0.28,0.3,0.32,0.31,3.41,0.0,-114.39,99.99,1.02,-6.7,1.1,-0.23,0.28,0.29,0.31,0.31,4.73,770.21,-69.63,100.86,11.89,1.44,4.69,2.14,0.3,0.31,0.32,0.31,0.01,33.47,9.83
                    1982-04-20,2.6,214.28,-70.91,100.98,7.59,1.72,0.04,2.36,0.27,0.28,0.31,0.31,1.73,0.0,-97.85,100.8,2.48,-0.14,-1.16,-0.83,0.27,0.27,0.31,0.31,3.41,629.34,-34.76,101.13,13.27,3.28,0.99,3.93,0.28,0.29,0.31,0.31,0.0,20.63,10.03
            下一天即1982-04-21的流量：8.58
            历史数据3：1982-04-22,0.76,185.81,-79.97,100.12,2.71,-7.65,3.75,-0.92,0.28,0.28,0.3,0.31,0.72,0.0,-102.11,99.91,-0.18,-10.02,1.89,-2.85,0.27,0.28,0.3,0.31,0.8,527.46,-48.72,100.52,6.03,-5.16,5.7,1.1,0.29,0.29,0.3,0.31,0.11,33.3,8.49
                    1982-04-23,0.67,237.54,-80.84,100.17,4.83,-2.37,2.37,0.77,0.26,0.27,0.29,0.31,0.52,0.0,-110.83,99.8,-1.35,-5.02,1.59,-1.12,0.26,0.26,0.29,0.31,0.78,779.73,-26.31,100.54,10.23,2.37,4.24,2.39,0.27,0.28,0.3,0.31,0.77,23.19,8.49
                    1982-04-24,0.38,260.83,-80.78,100.29,7.13,0.25,1.46,-0.98,0.27,0.26,0.29,0.31,0.29,0.0,-122.3,99.73,3.06,-3.88,0.37,-3.05,0.26,0.26,0.29,0.31,0.51,771.41,-15.9,100.77,12.53,3.89,2.23,1.98,0.28,0.27,0.29,0.31,2.19,24.3,7.33
                    1982-04-25,0.19,268.25,-81.56,100.56,11.23,5.0,1.43,2.57,0.25,0.26,0.28,0.31,0.08,0.0,-120.01,100.37,3.55,1.92,0.63,1.88,0.24,0.25,0.28,0.31,0.29,791.0,-3.6,100.78,19.2,7.68,2.25,3.06,0.26,0.26,0.29,0.31,0.0,26.07,6.08
                    1982-04-26,0.05,192.09,-57.36,100.83,11.66,6.63,1.5,3.02,0.24,0.25,0.28,0.31,0.02,0.0,-82.78,100.56,8.07,3.69,0.86,1.83,0.23,0.25,0.27,0.31,0.08,652.36,-12.63,101.07,16.29,8.99,1.87,4.24,0.26,0.25,0.28,0.31,2.45,20.41,5.37
                    1982-04-27,0.01,48.71,-8.07,100.37,10.17,9.85,0.48,2.68,0.31,0.28,0.27,0.31,0.0,0.0,-25.14,100.16,8.71,8.66,-0.76,-1.54,0.27,0.25,0.27,0.31,0.02,175.67,-0.47,100.9,11.96,10.74,1.96,5.28,0.36,0.3,0.27,0.31,17.3,2.28,6.08
                    1982-04-28,0.0,37.91,-10.12,100.65,4.45,1.45,-0.48,-4.01,0.3,0.29,0.27,0.31,0.0,0.0,-25.13,100.46,2.15,-2.62,-1.91,-4.78,0.29,0.29,0.27,0.31,0.0,162.91,1.18,100.75,9.19,7.41,1.7,-2.81,0.3,0.3,0.27,0.31,7.07,10.63,7.49
                    1982-04-29,0.0,287.58,-98.29,100.79,6.6,-6.25,0.72,-4.29,0.27,0.29,0.27,0.31,0.0,0.0,-125.93,100.69,2.81,-8.91,-0.69,-5.03,0.26,0.28,0.27,0.31,0.0,844.56,-25.09,100.94,11.47,-3.7,1.79,-3.49,0.29,0.29,0.27,0.31,0.01,47.41,6.95
                    1982-04-30,0.0,171.78,-71.66,100.26,4.25,-4.73,2.83,-2.11,0.26,0.27,0.27,0.31,0.0,0.0,-113.09,99.96,0.91,-8.78,1.57,-3.48,0.26,0.27,0.26,0.31,0.0,641.04,-37.65,100.75,8.37,-0.08,4.02,0.23,0.26,0.28,0.27,0.31,0.35,28.92,5.58
            下一天即1982-05-01的流量：4.5
        问题：{query}
        请根据资料内容进行推理作答，如资料不足，请说明不足之处。
    """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
        # max_new_tokens=max_new_tokens,
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 截掉 prompt，仅保留答案部分
    return answer[len(prompt):].strip()

# ========== 6. 示例调用 ==========
if __name__ == "__main__":
    q = "预测下一天的流量，给出推理过程和数值预测"
    ans = rag_ask(q, k=9, max_new_tokens=2048)
    print("📜 RAG 最终回答：")
    print(ans)

/Users/bierxiensi/Desktop/flood-rag/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bierxiensi/Desktop/flood-rag/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


正在加载 embedding 模型...
已向量化 9 条文档
正在加载 Qwen 模型...
模型加载完成


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
